In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from nn_parts import _mapping

import matplotlib.pyplot as plt
%matplotlib inline

# Summary

In this notebook I extract pretrained MobileNet weights from the Tensorflow checkpoint downloaded here:
https://github.com/tensorflow/models/blob/master/slim/nets/mobilenet_v1.md

# Graph

In [3]:
tf.reset_default_graph()
optimizer = tf.train.RMSPropOptimizer(1e-3)
num_classes = 1001

X = tf.placeholder(tf.float32, [None, 224, 224, 3], 'X')
Y = tf.placeholder(tf.float32, [None, num_classes], 'Y')

with tf.variable_scope('control'):
    is_training = tf.placeholder_with_default(True, [], 'is_training')

with tf.variable_scope('MobilenetV1'):
    logits = _mapping(X, num_classes, is_training)

with tf.variable_scope('log_loss'):
    log_loss = tf.losses.softmax_cross_entropy(Y, logits)

with tf.variable_scope('total_loss'):
    total_loss = tf.losses.get_total_loss()

ema = tf.train.ExponentialMovingAverage(decay=0.99)
maintain_averages_op = ema.apply(
    tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimize_op = optimizer.minimize(total_loss)

with tf.control_dependencies([optimize_op]):
    optimize = tf.group(maintain_averages_op)

with tf.variable_scope('utilities'):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

# Session

In [4]:
gv = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
weights = [
    v for v in gv 
    if ('ExponentialMovingAverage' in v.name) or ('moving_' in v.name)
]
extracted_weights = {}

In [5]:
sess = tf.Session()
saver.restore(sess, 'pretrained_model/mobilenet_v1_1.0_224.ckpt')
for tensor in weights:
    tensor_value = sess.run(tensor)
    
    if 'ExponentialMovingAverage' in tensor.name:   
        name = tensor.name[:-27] + ':0'
    elif 'moving_' in tensor.name:
        name = tensor.name
    
    extracted_weights[name] = tensor_value

sess.close()

INFO:tensorflow:Restoring parameters from pretrained_model/mobilenet_v1_1.0_224.ckpt


In [6]:
np.save('extracted_weights.npy', extracted_weights)